# Exercise set 1

In [8]:
from utils import ListNode

## 5. 最长回文子串

[力扣](https://leetcode.cn/problems/longest-palindromic-substring/)

### 方法一: 动态规划

很容易得知，状态转移方程为:

  $$dp[i][j] = (s[i]==s[j]) \land dp[i-1][j+1]$$

但是关键在于，如何 sweep 整个 dp 空间呢？从状态转移方程中，我们可以看到，dp 的依赖关系是：长子串依赖于短子串。因此我们的遍历方式选择从子串长度入手！大致思路为:
```python
# dp init
# calculate dp elements for L = 1, i.e. dp[i][i] for i in range(len(s))

# dp sweep
for L in range(2, len(s)+1):
    # calcualte dp elements for substring with length `L`
    for i in range(len(s) - L):
        j = i + L - 1 # because L = j - i + 1
        pass
```

In [9]:
def longestPalindrome(s):
    n = len(s)

    # dp init for L = 1
    dp = [[False] * n for _ in range(n)]
    for i in range(n):
        dp[i][i] = True
    sub_str = s[0]

    # dp sweep, L --> L-2, so we iterate from L = 2...n
    # dp[i][j] = s[i] == s[j] && s[i+1][j-1]
    for L in range(2, n+1):
        cnt = False
        for i in range(n+1-L):
            # i = 0... n-L, j = L-1...n-1
            j = i + L - 1  # L = j - i + 1
            dp[i][j] = (s[i] == s[j])
            if L > 2:
                dp[i][j] = dp[i][j] and dp[i+1][j-1]
            # update max_len
            if not cnt and dp[i][j]:
                cnt = True
                sub_str = s[i:j+1]
    return sub_str

print("expected: bab")
longestPalindrome("babad")

expected: bab


'bab'

### 方法二: 中心扩散

仔细观察动态规划的状态转移方程，我们会发现，dp 依赖不仅有： 长串依赖短串的规律，还有这里的长短串是相同中心的！

因此，我们完全可以改变刚才 dp sweep 的内外循环顺序。先遍历各个点作为回文中心，再逐渐增加长度。

In [10]:
# center_expand 不容易写对
def center_expand(s, l, r):
    while l>=0 and r < len(s) and s[l] == s[r]:
        l -= 1
        r += 1
    return l+1, r-1

def longestPalindrome(s):
    max_len = 0
    sub_str = ""
    for i in range(len(s)):
        # 以 s[i] 为中心，尝试中心拓展
        l1, r1 = center_expand(s, i, i)
        l2, r2 = center_expand(s, i, i+1)
        if r1 - l1 + 1 > max_len:
            max_len, sub_str = (r1-l1+1), s[l1:r1+1]
        if r2 - l2 + 1 > max_len:
            max_len, sub_str = (r2-l2+1), s[l2:r2+1]
    return sub_str

print("expected: a")
longestPalindrome("a")

expected: a


'a'

## 6. N 字形变换

思路还是找规律。

`row` = 2:

0|2|4|6
1|3|5|7

`row` = 3:
0| |4| |8|
1|3|5|7|9|
2| |6| |



In [11]:
def convert( s, numRows):
    """
    :type s: str
    :type numRows: int
    :rtype: str
    """
    if numRows == 1:
        return s
    # numRows >= 2
    ans = []
    margin = 2 * (numRows - 1)

    for row in range(0, numRows):
        i = row
        sub_margin = row * 2
        odd = False
        while i < len(s):
            ans.append(s[i])
            if row == 0 or row == numRows-1:
                i += margin
            else:
                if odd:
                    i += sub_margin
                else:
                    i += (margin - sub_margin)
            odd = not odd
    return "".join(ans)

print("expected result: PAHNAPLSIIGYIR")
convert("PAYPALISHIRING", 3)

expected result: PAHNAPLSIIGYIR


'PAHNAPLSIIGYIR'

## 7. 整数反转

这道题不难，关键在于边界条件能不能按照题目要求处理好。

题目要求：假设环境中不能存储 64 位正数，若反转后值溢出，则返回 0 . 通常处理溢出，我们可以通过结果变号来识别，但是事实是：我们的环境可以存储 64 位整数，因此我们无法通过这个结果变号来判断，而是手工判断。

In [12]:
def reverse(x):
    """
    :type x: int
    :rtype: int
    """
    # 1. save sign, convert positive for convinience
    INT_MIN, INT_MAX = -2**31, 2**31-1
    positive = (x>0)
    x = x if positive else (-x)

    # 2. reverse
    stack = []
    base = 1
    result = 0
    while x > 0:
        stack.append(x % 10)
        x //= 10

    for i in range(len(stack)):
        if positive and result > INT_MAX // 10:
            return 0
        if not positive and (-result < INT_MIN // 10 + 1):
            return 0
        result = 10 * result + stack[i]

    # handle overflow, we have to notice that: we are pretending that the env cannot save 64-bit integer while it can
    # so, the following check is actually a cheat
    # if result >> 31 != 0:
    #     return 0

    return result if positive else (-result)

print('expected result: -392175')
reverse(-571293)

expected result: -392175


-392175

## 2. 两数相加

这道题简单，但是非常考察细心程度。

1. 要使用同样逆序的链表返回
3. 要注意 `l1`, `l2` 都到达末尾后，仍存在的进位！！

In [13]:


def addTwoNumbers(self, l1, l2):
    """
    :type l1: ListNode
    :type l2: ListNode
    :rtype: ListNode
    """
    head = ListNode()
    cur_node = head
    outflow = 0
    while l1 or l2:
        digit_1 = int(l1.val) if l1 else 0
        digit_2 = int(l2.val) if l2 else 0
        digit_sum = digit_1 + digit_2 + outflow
        outflow = digit_sum // 10
        cur_node.next = ListNode(digit_sum % 10)
        cur_node = cur_node.next
        if l1:
            l1 = l1.next
        if l2:
            l2 = l2.next
    if outflow > 0:
        cur_node.next = ListNode(outflow)
    return head.next

## 1. 两数之和

关键点: 使用 hash 结构，空间换时间

### 版本一: 返回一组解(下标)

In [14]:
def two_sum(nums, target):
    cache = {}
    for i, num in enumerate(nums):
        if num in cache:
            return {cache[num], i}
        else:
            cache[target-num] = i

two_sum([2, 7, 12, 2, 12, 11, 15, 1,5, 14, 3, 3, 4, 13], 14)

{0, 2}

### 版本二: 返回所有解(下标)

In [15]:
def two_sum(nums, target):
    from collections import defaultdict
    ans = []
    cache = defaultdict(lambda: [])
    for i, num in enumerate(nums):
        if num in cache:
            for idx in cache[num]:
                ans.append([idx, i])
        cache[target-num].append(i)
    return ans

two_sum([2, 7, 12, 2, 12, 11, 15, 1, 5, 14, 3, 3, 4, 13], 14)

[[0, 2], [2, 3], [0, 4], [3, 4], [5, 10], [5, 11], [7, 13]]

### 版本三: 返回所有解(值)

In [16]:
def two_sum(nums, target):
    from collections import defaultdict
    ans = []
    cache = defaultdict(lambda: [])

    for i, num in enumerate(nums):
        if num in cache:
            for idx in cache[num]:
                ans.append([idx, i])
        cache[target-num].append(i)

    ans = [[nums[pair[0]], nums[pair[1]]] for pair in ans]
    return ans

two_sum([2, 7, 12, 2, 12, 11, 15, 1, 5, 14, 3, 3, 4, 13], 14)

[[2, 12], [12, 2], [2, 12], [2, 12], [11, 3], [11, 3], [1, 13]]

## 15. 三数之和

[力扣](https://leetcode-cn.com/problems/3sum/)

三数之和与两数之和的问题完全不同： 排序 + 双指针

关键点： 先排序，再借助双指针。将 $O(N^3) $ 转化为 $O(N^2)$

### 方法一: 有序枚举

In [17]:
def threeSum(nums):
    ans = []
    nums = sorted(nums)
    n = len(nums)

    # first, second, third
    # given first, if second increase, third must decrease
    # iterate first over 0...n-3
    for first in range(n-2):
        # guarentee non-duplicity of first
        if first > 0 and nums[first] == nums[first-1]:
            continue

        third = n-1
        # iterate second over first+1...n-2
        for second in range(first+1, n-1):
            # guarentee non-duplicity of second:
            if second > first+1 and nums[second] == nums[second-1]:
                continue
            # find the proper third
            while second < third and nums[first] + nums[second] + nums[third] > 0:
                third -= 1
            # if end while because of finding no solution
            if second == third:
                continue
            # if finding a correct solution, add the solution to solution set
            if nums[first] + nums[second] + nums[third] == 0:
                ans.append([nums[first], nums[second], nums[third]])
    return ans


### 方法二: 双指针法

In [18]:
def threeSum(nums):
        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        ans = []
        nums = sorted(nums)
        n = len(nums)

        # iterate first over 0...n-3
        for first in range(n-2):
            # guarentee non-duplicity of first
            if first > 0 and nums[first] == nums[first-1]:
                continue

            # find all proper second and third by two pointers methods
            L, R = first+1, n-1
            while L < R:
                temp_sum = nums[first] + nums[L] + nums[R]
                if temp_sum == 0:
                    ans.append([nums[first], nums[L], nums[R]])
                    while L<R and nums[L] == nums[L+1]: L += 1
                    while L<R and nums[R] == nums[R-1]: R -= 1
                    L, R = L+1, R-1
                elif temp_sum > 0:
                    R -= 1
                else:  # temp_sum < 0
                    L += 1
        return ans

## 22. 括号生成

[力扣](https://leetcode-cn.com/problems/generate-parentheses/)

### 回溯法

In [19]:
class Solution():
    ans = []

    def generate_parenthesis(self, n):
        self.ans = []
        self.backtrace("", n, n)
        return self.ans

    def backtrace(self, s, bra, ket):
        if bra  == 0 and ket == 0:
            self.ans.append(s)
            return
        # remaining bra must <= ket
        if bra == ket:
            self.backtrace(s + "(", bra-1, ket)
        elif bra < ket:
            if bra > 0:
                self.backtrace(s + "(", bra-1, ket)
            self.backtrace(s + ")", bra, ket-1)


Solution().generate_parenthesis(4)

['(((())))',
 '((()()))',
 '((())())',
 '((()))()',
 '(()(()))',
 '(()()())',
 '(()())()',
 '(())(())',
 '(())()()',
 '()((()))',
 '()(()())',
 '()(())()',
 '()()(())',
 '()()()()']

## 21. 归并 2 个有序链表

[力扣](https://leetcode-cn.com/problems/merge-two-sorted-lists/)

### 方法一: 递归

In [20]:
# Definition for singly-linked list.
# class ListNode(object):
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next
class Solution(object):
    def mergeTwoLists(self, list1, list2):
        """
        :type list1: Optional[ListNode]
        :type list2: Optional[ListNode]
        :rtype: Optional[ListNode]
        """
        if list1 is None:
            return list2
        if list2 is None:
            return list1

        if list1.val <= list2.val:
            list1.next = self.mergeTwoLists(list1.next, list2)
            return list1
        else:
            list2.next = self.mergeTwoLists(list1, list2.next)
            return list2

### 方法二: 迭代

In [21]:
# Definition for singly-linked list.
# class ListNode(object):
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next
class Solution(object):
    def mergeTwoLists(self, list1, list2):
        """
        :type list1: Optional[ListNode]
        :type list2: Optional[ListNode]
        :rtype: Optional[ListNode]
        """
        head = ListNode()
        cur_node = head
        while list1 is not None or list2 is not None:
            if list1 is None:
                cur_node.next = list2
                break
            if list2 is None:
                cur_node.next = list1
                break
            if list1.val <= list2.val:
                cur_node.next = ListNode(list1.val)
                list1 = list1.next
            else:
                cur_node.next = ListNode(list2.val)
                list2 = list2.next
            cur_node = cur_node.next
        return head.next

## 23. 归并 K 个有序链表

[力扣](https://leetcode-cn.com/problems/merge-k-sorted-lists/)

中心思想： 二分，分而治之

### 方法一: 递归

In [22]:
def mergeKList(linked_list:list):
    n = len(linked_list)
    if n == 0:
        return []
    if n == 1:
        return linked_list[0]
    if n == 2:
        return merge2List(linked_list[0], linked_list[1])




def merge2List(node_a:ListNode, node_b:ListNode):
    if node_a is None:
        return node_b
    if node_b is None:
        return node_a
    if node_a.val <= node_b.val:
        node_a.next = merge2List(node_a.next, node_b)
        return node_a
    else:
        node_b.next = merge2List(node_a, node_b.next)
        return node_b


### 方法二: 迭代

## 25. K个一组翻转链表

[力扣](https://leetcode-cn.com/problems/reverse-nodes-in-k-group/)

### 方法一：递归

In [23]:
# Definition for singly-linked list.
# class ListNode(object):
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next
class Solution(object):
    def reverseKGroup(self, head, k):
        """
        :type head: ListNode
        :type k: int
        :rtype: ListNode
        """
        # 0. 特殊情况
        if k == 1:
            return head
        # 现在 k >= 2
        i = k
        next_start = head
        # 1. 检查有没有 K 个
        while i > 1 and next_start is not None:
            next_start = next_start.next
            i -= 1
        # 1.(1) 如果没有，则直接返回
        if next_start is None:
            return head
        # 2.(2) 如果有, 则开始反转
        next_start = next_start.next  # next_start 指向下一组开始
        prev, cur, nex = None, head, head.next
        while cur != next_start:
            nex = cur.next
            cur.next = prev
            prev, cur = cur, nex
        # 循环结束时, prev 指向反转后列表的新头节点, head 现在指向第一组的尾节点
        head.next = self.reverseKGroup(next_start, k)
        return prev


### 方法二: 迭代

编写一个 `reverse` 函数，返回反转后的链表头节点 `head` 和尾节点 `tail`，然后在 `reverseKGroup` 中调用 `reverse` 函数即可。

主要是维护下面几个指针:
- `pre` 指向上一组的最后一个元素
- `head` 指向当前组的第一个元素
- `tail` 指向当前组的最后一个元素
- `next_start` 指向下一组的第一个元素

In [24]:
class Solution:
    # 翻转从 head 至 tail 的链表，返回新的头尾节点
    def reverse(self, head, tail):
        pre, cur = None, head
        while cur and pre!=tail:
            nex = cur.next
            cur.next = pre
            pre, cur = cur, nex
        return tail, head

    def reverseKGroup(self, head, k):
        hair = ListNode()
        hair.next = head
        pre = hair  # pre 指向上一组的最后一个元素

        while head:
            tail = pre  # tail 要指向当前组的最后一个元素
            for i in range(k):
                tail = tail.next
                if tail is None:
                    return hair.next
            next_start = tail.next  # next_start 指向下一组的第一个元素
            head, tail = self.reverse(head, tail)  # 调用时，while 可以保证 head 不为空, for 可以保证 tail 不为空
            pre.next = head
            tail.next = next_start
            pre, head = tail, next_start
        return hair.next


## 8. atoi

[力扣](https://leetcode.cn/problems/string-to-integer-atoi/)

### 解法一: 细心即可

In [25]:
class Solution(object):
    INT_MAX = 2**31-1
    INT_MIN = -2**31

    def myAtoi(self, s):
        """
        :type s: str
        :rtype: int
        """
        # step 1
        s = s.strip()
        n = len(s)
        num = 0

        # special case
        if n == 0:
            return 0

        i = 0
        is_neg = False
        # step 2:
        if s[i] == '-':
            is_neg = True
            i += 1
        elif s[i] == '+':
            i += 1

        # step 3: ignore all prior 0
        while i<n and ord(s[i])==ord('0'):
            i += 1

        # read until end or non-numeric element
        digits = []
        while i<n and ord(s[i])<=ord('9') and ord(s[i])>=ord('0'):
            digits.append(ord(s[i]) - ord('0'))
            i += 1

        # step 4, 5: convert digits to an integer
        for digit in digits:
            # positive overflow
            if not is_neg and (num>self.INT_MAX//10 or (num==self.INT_MAX//10 and digit>=8)):
                return self.INT_MAX
            if is_neg and (num>(-self.INT_MIN)//10 or (num==(-self.INT_MIN)//10 and digit>=8)):
                return self.INT_MIN
            num = num * 10 + digit

        return -num if is_neg else num

### 解法二: 有限状态机 DSM

## 17. 电话号码的字母组合

显然，可以使用回溯法，还不用剪枝。

### 解法一: 回溯法

denote `len(digits)` as $n$

时间复杂度: $O(3^n)$
空间复杂度: $O(3^n)$

> p.s. 本身正好有 $3^n$ 种组合。因此这个时空复杂度是 ok 的。

In [26]:
class Solution(object):
    ans = []
    num2char = {
            '2': ['a', 'b', 'c'],
            '3': ['d', 'e', 'f'],
            '4': ['g', 'h', 'i'],
            '5': ['j', 'k', 'l'],
            '6': ['m', 'n', 'o'],
            '7': ['p', 'q', 'r', 's'],
            '8': ['t', 'u', 'v'],
            '9': ['w', 'x', 'y', 'z']
    }
    def letterCombinations(self, digits):
        """
        :type digits: str
        :rtype: List[str]
        """
        # 这显然是不剪枝的回溯
        self.ans = []
        if len(digits) > 0:
            self.get_combination(digits, "")
        return self.ans

    def get_combination(self, digits, s):
        if len(digits) == 0:
            self.ans.append(s)
            return
        # len(digits) >= 1
        for c in self.num2char[digits[0]]:
            self.get_combination(digits[1:], s+c)
        return

### 解法二: 队列



In [27]:
class Solution(object):
    num2char = {
            '2': ['a', 'b', 'c'],
            '3': ['d', 'e', 'f'],
            '4': ['g', 'h', 'i'],
            '5': ['j', 'k', 'l'],
            '6': ['m', 'n', 'o'],
            '7': ['p', 'q', 'r', 's'],
            '8': ['t', 'u', 'v'],
            '9': ['w', 'x', 'y', 'z']
    }
    def letterCombinations(self, digits):
        """
        :type digits: str
        :rtype: List[str]
        """
        # 再来尝试队列
        if len(digits) == 0:
            return []
        queue = [""]

        for digit in digits:
            cur = []
            while len(queue) > 0:
                cur.append(queue.pop(0))
            for s_prev in cur:
                for c in self.num2char[digit]:
                    queue.append(s_prev + c)
        return queue

## 33. 搜索旋转排序数组

二分查找的进阶版（尚未解决）

## 86 分隔链表

[力扣](https://leetcode.cn/problems/partition-list/)

简单分割，算法含量较低

In [28]:
# Definition for singly-linked list.
# class ListNode(object):
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next
class Solution(object):
    def partition(self, head, x):
        """
        :type head: ListNode
        :type x: int
        :rtype: ListNode
        """
        # special case
        if head is None or head.next is None:
            return head

        # 2 lists
        sm_list, lg_list = ListNode(), ListNode()
        sm_cur, lg_cur = sm_list, lg_list

        while head:
            if head.val < x:
                sm_cur.next = ListNode(head.val)
                sm_cur = sm_cur.next
            else:
                lg_cur.next = ListNode(head.val)
                lg_cur = lg_cur.next
            head = head.next
        sm_cur.next = lg_list.next
        return sm_list.next


## 142. 环形链表 2

[力扣](https://leetcode.cn/problems/linked-list-cycle-ii/)

写出数学公式，寻找相遇方法

In [29]:
# Definition for singly-linked list.
# class ListNode(object):
#     def __init__(self, x):
#         self.val = x
#         self.next = None

class Solution(object):
    def detectCycle(self, head):
        """
        :type head: ListNode
        :rtype: ListNode
        """
        if head is None:
            return None

        slow, fast = head, head.next

        while slow and fast:
            if slow == fast:
                # 找到一个交点，则进一步找入口
                slow = slow.next
                while slow != head:
                    slow = slow.next
                    head = head.next
                return slow
            slow = slow.next
            fast = fast.next
            if fast:
                fast = fast.next
            else:
                return None
        return None

### 92. 反转链表 II

[力扣](https://leetcode.cn/problems/reverse-linked-list-ii/)

In [30]:
# Definition for singly-linked list.
# class ListNode(object):
#     def __init__(self, val=0, next=None):
#         self.val = val
#         self.next = next
class Solution(object):
    def reverseBetween(self, head, left, right):
        """
        :type head: ListNode
        :type left: int
        :type right: int
        :rtype: ListNode
        """
        # 1. Find left
        prev_left, cur_left = None, head
        l, r = left, right
        while cur_left and l > 1:
            prev_left = cur_left
            cur_left = cur_left.next
            l -= 1
            r -= 1

        if cur_left is None:
            return head

        # 2. Left is not None, than start inversion
        prev, cur, nex = prev_left, cur_left, cur_left.next
        while cur and r >= 1:
            nex = cur.next
            cur.next = prev
            prev, cur = cur, nex
            r -= 1

        # prev 指向 right, cur_left 现在是 尾部
        cur_left.next = cur
        if left == 1:
            return prev
        else:
            prev_left.next = prev
            return head

## 739. 每日温度

[力扣](https://leetcode.cn/problems/daily-temperatures/)

方法一: 栈

In [31]:
class Solution(object):
    def dailyTemperatures(self, temperatures):
        """
        :type temperatures: List[int]
        :rtype: List[int]
        """
        stack = [[0,temperatures[0]]]
        answer = [0] * len(temperatures)
        for i, temp in enumerate(temperatures):
            if len(stack) == 0 or temp <= stack[-1][1]:
                stack.append([i, temp])
            else:  # 更高的气温出现了
                while len(stack)>0 and stack[-1][1] < temp:
                    j, _ = stack.pop()
                    answer[j] = (i-j)
                stack.append([i, temp])
        return answer

### 方法二: 动态规划

## 28. 实现 strStr()

[力扣](https://leetcode-cn.com/problems/implement-strstr/)

### 方法一: 暴力法

In [32]:
class Solution(object):
    def strStr(self, haystack, needle):
        """
        :type haystack: str
        :type needle: str
        :rtype: int
        """
        m, n = len(haystack), len(needle)
        if m < n:
            return -1

        for i in range(m-n+1):
            # i = 0...m-n
            solved = True
            for j in range(n):
                # j = 0...n-1
                if haystack[i+j] != needle[j]:
                    solved = False
            # 匹配成功
            if solved:
                return i
        return -1

### 方法二: KMP

参考知乎“如何更好地理解和掌握KMP算法”的[回答](https://www.zhihu.com/question/21923021/answer/281346746)

KMP 算法的核心思想是：**如果存在公共前后缀，那么就可以跳步匹配**，实现这一思想的核心在于 PMT 表(Partial Match Table)。

因此，理解 KMP 算法，也就是需要理解两个问题：

1. 什么是PMT 表
2. 如何利用 PMT 表进行匹配

首先，我们看看什么是 PMT 表。

#### 1. 什么是 PMT 表

PMT 表的含义是：对于字符串的每一个位置，从 0 到当前位置的子串的最长公共前后缀的长度。

假设我们有一个字符串 `abababca`，那么它的 PMT 表如下：

![](../imgs/PMT.png)

例如 index 为 2 的位置，对应的子串是 `aba`，它的最长公共前后缀是 `a`，长度为 1。而 index 为 5 的位置，对应的子串是 `ababa`，它的最长公共前后缀是 `aba`，长度为 3。

#### 2. 如何利用 PMT 表进行匹配

abcdabcdcababab

abcdabcdab

i 不动, j 从 index 变化到 pmt 位置与 i 进行比较

i = 0...n-1
j=0
hay[i] 与 needle[j] 匹配：
若匹配成功：
  则 i+=1, j += 1
若匹配失败：
  则 j = next[j]
  若 j == -1, 则 continue

In [33]:
class Solution(object):
    def strStr(self, haystack, needle):
        """
        :type haystack: str
        :type needle: str
        :rtype: int
        """
        m, n = len(haystack), len(needle)
        if m < n:
            return -1

        # a b c a b a b a
        # 构建 next 数组
        pmt = [0] * n
        for i in range(1, n):
            # calculate pmt[i]
            last_pmt = pmt[i-1]
            if needle[last_pmt] == needle[i]:
                pmt[i] = pmt[i-1] + 1
            else:
                pass


        # 利用 next 数组进行匹配
        i, j = 0, 0
        while i<m and j<n:
            if j==-1 or haystack[i]==needle[j]:
                i += 1
                j += 1
            else:
                j = next[j]

        if j == n:
            return i-j
        else:
            return -1


## 50. Pow(x, n)

[力扣](https://leetcode-cn.com/problems/powx-n/)

**思路**

先算出 `powers` 数组, 也就是需要的 $[x^0, x^1, x^2, ...]$ 数组
然后再基于这个数组去算。

In [34]:
class Solution(object):
    def myPow(self, x, n):
        """
        :type x: float
        :type n: int
        :rtype: float
        """
        # 基础情况
        if n == 0:
            return 1

        is_neg = n < 0
        abs_n = n if n >= 0 else -n

        res = 1
        i = 0
        powers = [1, x]
        while i < abs_n:
            if i == 0:
                res *= x
                i += 1
            else:  # i > 0
                if i * 2 <= abs_n:
                    res *= res
                    i = i*2
                    powers.append(res)
                else:  # 进入收尾阶段，最大的 power 已经算出来了
                    j = len(powers)-2
                    i_inc = i // 2
                    while i < abs_n:
                        if i+i_inc>abs_n:
                            j = j-1
                            i_inc = i_inc // 2
                            continue
                        # i+i_inc<=abs_n
                        res *= powers[j]
                        i += i_inc

        return 1./res if is_neg else res


## 有效三角形的个数

[力扣](https://leetcode-cn.com/problems/valid-triangle-number/)

**思路**

排序，双指针

In [35]:
class Solution(object):
    def triangleNumber(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        num_tris, n = 0, len(nums)
        nums.sort()

        for i in range(n-2):
            if nums[i] == 0:
                continue
            k = i + 2  # k 是第一个不能组成三角形的下标, 范围: i+2...n
            for j in range(i+1, n-1):
                while k<n and nums[i]+nums[j]>nums[k]:
                    k += 1
                num_tris += (k-j-1)  # e.g. 2, 2, 3, 3, i=0, j=1, 共有 2 种, k=4, k-j-1 才对
        return num_tris

稍微改变写法, 提高效率

给定 right 和 mid, 让 left 向右边凑。这种做法利用了: nums[left] + nums[middle] > nums[right] 的对称性！

我们应当选择 `left` 和 `middle` 作为双指针, 才可以利用对称性！！

AC 时间从 1000 ms 骤降至 500 ms

In [36]:
class Solution(object):
    def triangleNumber(self, nums):
        nums.sort()
        n = len(nums)
        res = 0
        for right in range(2, n):
            left = 0
            mid = right - 1
            while left < mid:
                if nums[left] + nums[mid] > nums[right]:
                    res += mid - left
                    mid -= 1
                else:
                    left += 1
        return res

## 罗马转数字

[力扣 13](https://leetcode.cn/problems/roman-to-integer/)

### 方法一: 使用优先列表比对

![](../imgs/13_naive.png)

In [37]:
class Solution(object):
    def romanToInt(self, s):
        """
        :type s: str
        :rtype: int
        """
        rom_2_num = (
            ['IV',   4], ['IX',   9], ['I',   1], ['V',   5],
            ['XL',  40], ['XC',  90], ['X',  10],['L',  50],
            ['CD', 400], ['CM', 900], ['C', 100],['D', 500],
            ['M', 1000]
        )



        i, n, res = 0, len(s), 0
        while i < n:
            for rom, val in rom_2_num:
                if len(rom) <= n-i and rom == s[i:i+len(rom)]:
                    res += val
                    i += len(rom)
                    break
        return res

### 方法二: 直接求值

![](../imgs/13.png)

In [38]:
class Solution(object):
    def romanToInt(self, s):
        """
        :type s: str
        :rtype: int
        """
        rom_2_val = {
            'I':1, 'V':5, 'IV':4, 'X':10, 'IX': 9,
            'L':50, 'C': 100, 'XL': 40, 'XC': 90,
            'D':500, 'M':1000, 'CD':400, 'CM':900
        }

        match = {'I': ['V', 'X'], 'X': ['L', 'C'], 'C': ['D', 'M']}

        i, res, n, val = 0, 0, len(s), 0

        while i < n:
            if (s[i] in match) and i+1<n:
                if s[i+1] in match[s[i]]:
                    val = rom_2_val[s[i:i+2]]
                    i += 2
                    res += val
                    continue
            val = rom_2_val[s[i]]
            i += 1
            res += val
        return res

## 数字转罗马

[力扣 12](https://leetcode-cn.com/problems/integer-to-roman/)

## 方法一: 优先列表比对

![](../imgs/12.png)

In [39]:
class Solution(object):
    romans = [
        ['I', 'V', 'X'],
        ['X', 'L', 'C'],
        ['C', 'D', 'M'],
        ['M', 'M', 'M']
    ]

    def intToRoman(self, num):
        """
        :type num: int
        :rtype: str
        """
        res = ""
        i = 0
        while num > 0:
            digit = num % 10
            num = num // 10
            res = self.digitToRoman(digit, self.romans[i]) + res
            i+=1
        return res

    def digitToRoman(self, digit, roms):
        # roms = ['I', 'V', 'X'], digit = 0...9
        if digit < 4:
            return roms[0] * digit
        elif digit == 4:
            return roms[0] + roms[1]
        elif digit == 5:
            return roms[1]
        elif digit < 9:
            return roms[1] + (roms[0] * (digit-5))
        else:
            return roms[0] + roms[2]

### 解法二: 使用哈希表

**注意**: 从 Python 3.6 之后, 字典的顺序是有序的, 但是在 Python 3.5 之前, 字典的顺序是无序的。因此, 该解法只试用于 Python 3.6 之后, i.e. Leetcode 的 Python3 模式。

Python3 的字典是由 hashTable 来实现的, 相比于 Python2, 引入了额外的 indices 结构, 因此会造成额外的空间开销。

![](../imgs/12_hashtable.png)

In [40]:
class Solution:
    def intToRoman(self, num: int) -> str:
        # 使用哈希表，按照从大到小顺序排列
        hashmap = {1000:'M', 900:'CM', 500:'D', 400:'CD', 100:'C', 90:'XC', 50:'L', 40:'XL', 10:'X', 9:'IX', 5:'V', 4:'IV', 1:'I'}
        res = ''
        for key in hashmap:
            if num // key != 0:
                count = num // key  # 比如输入4000，count 为 4
                res += hashmap[key] * count
                num %= key
        return res

## 外观数列

[力扣 38](https://leetcode-cn.com/problems/count-and-say/)

![](../imgs/38_naive.png)

In [41]:
class Solution:
    def countAndSay(self, n: int) -> str:
        s = "1"
        while n > 1:
            # 1. 构建 num_count
            num_count = []
            i, num, count = 1, s[0], 1
            while i<=len(s):
                while i<len(s) and s[i]==s[i-1]:
                    count += 1
                    i += 1
                num_count.append([num, count])
                if i >= len(s):
                    break
                num, count = s[i], 1
                i += 1
            # 2. 根据 num_count 构建 s
            s = "".join([str(pair[1]) + pair[0]  for pair in num_count])
            n -= 1
        return s

### 官方题解: 更整洁

![](../imgs/38.png)

In [42]:
def countAndSay(self, n: int) -> str:
        prev = "1"
        for i in range(n-1):
            curr = ""
            pos = 0
            start = 0
            while pos < len(prev):
                while pos < len(prev) and prev[pos] == prev[start]:
                    pos += 1
                curr += str(pos - start) + prev[start]
                start = pos
            prev = curr
        return prev

## 加一

[力扣 66](https://leetcode-cn.com/problems/plus-one/)

![](../imgs/66.png)

In [44]:
class Solution:
    def plusOne(self, digits):
        carry,already_add = 0, False
        for i in range(len(digits)-1, -1, -1):
            digit = digits[i] + carry
            if not already_add:
                digit += 1
                already_add = True
            carry = digit // 10
            digits[i] = digit % 10

        if carry > 0:
            digits = [carry] + digits

        return digits
